# Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dependencies

In [ ]:
#Dependencies to load the dataset in tf.data format
!pip install med_dataloader==0.1.12 -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 15.4 MB/s eta 0:00:00


In [ ]:
#Link colab to the directories of a given google drive account
import os
from google.colab import drive
ROOT_PATH = os.path.join(os.sep, 'content' , 'gdrive')
drive.mount(ROOT_PATH)

Mounted at /content/gdrive


In [ ]:
ROOT_PATH

'/content/gdrive'

In [ ]:
#Possible usefull dependencies
import med_dataloader as mdl
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import Model
import keras

from datetime import datetime
from dateutil.tz import gettz
import SimpleITK as sitk

import scipy
import cv2

import SimpleITK as sitk
import imageio

from sklearn.utils import shuffle
import re

In [ ]:
WD_PATH = os.path.join(ROOT_PATH, 'MyDrive', 'PROJECT_NEURO')

DATASET_PATH = os.path.join(WD_PATH, 'Dataset_GS') #Move the folder containig dataset (e.g. Dataset_SP) into WD_PATH

MODELS_PATH = os.path.join(WD_PATH, 'Models') #Folder where best trained models will be saved

if not os.path.exists(MODELS_PATH): #si può togliere
    os.mkdir(MODELS_PATH)


Uncached Data

In [ ]:
# lisa di unchached images' file name
images = os.listdir('/content/gdrive/MyDrive/PROJECT_NEURO/GS_images')

In [ ]:
# exctract the subjects
subjects = set([i[:6] for i in images])
subjects = sorted(list(subjects))
subjects

['s00001', 's00002', 's00004', 's00005']

In [ ]:
# numero di immagini per ciascun paziente
# Inizializza un dizionario per tenere traccia del numero di immagini per soggetto
images_for_subjects_dict = {}

# Conta il numero di immagini per ciascun soggetto
for image in images:
    subject = image[:6]
    images_for_subjects_dict[subject] = images_for_subjects_dict.get(subject, 0) + 1

# Se necessario, converte il dizionario in una lista ordinata di conteggi
images_for_subjects = [images_for_subjects_dict.get(subject, 0) for subject in subjects]

print(images_for_subjects)


[280, 240, 150, 150]


# Data Loader
carico i dati da Subject divided e poi stratifico per i diversi pazienti

In [ ]:
uncached=True

#parameters as global variables
seed=42
NUM_CLASSES = 7
BATCH_SIZE = 8  #questo dopo lo possiamo modificare per migliorare il training
EPOCHS = 70     #questo lo possiamo modificare se il training si ferma perchè raggiunge il massimo di epochs

#Percentages for dataset splitting
PERC_TRAIN_DATA = 0.7               #Percentage of the total dataset that will be used for training --> backpropagation and updating of weights and bias are based only on this data
PERC_VALIDATION_DATA = 0.2          #Percentage of the total dataset that will be used during training to evaluate if the Net is overfitting
PERC_TEST_DATA = 0.1                #Percentage of the total dataset that will be used after training to evaluate the global Net performances


if uncached:
  #nostro caricamento dei dati
  total=sum(images_for_subjects)
  unique_gray_levels = np.linspace(0, 6, 7)

  # Lists to store images and labels
  X_train_list, y_train_list = [], []
  X_val_list, y_val_list = [], []
  X_test_list, y_test_list = [], []

  i1=int(images_for_subjects[0]*PERC_TRAIN_DATA)
  ii1=int(images_for_subjects[0]*(PERC_TRAIN_DATA+PERC_VALIDATION_DATA))
  iii1=images_for_subjects[0]

  i2=iii1+int(images_for_subjects[1]*PERC_TRAIN_DATA)
  ii2=i2+int(images_for_subjects[1]*PERC_VALIDATION_DATA)
  iii2=iii1+images_for_subjects[1]

  i3=iii2+int(images_for_subjects[2]*PERC_TRAIN_DATA)
  ii3=i3+int(images_for_subjects[2]*PERC_VALIDATION_DATA)
  iii3=iii2+images_for_subjects[2]

  i4=iii3+int(images_for_subjects[3]*PERC_TRAIN_DATA)
  ii4=i4+int(images_for_subjects[3]*PERC_VALIDATION_DATA)
  iii4=total

  directory_path = '/content/gdrive/MyDrive/PROJECT_NEURO/Subject divided' #load only one image to decide how to preprocces
  label_directory_path= '/content/gdrive/MyDrive/PROJECT_NEURO/Segmentation_checked'
  i=-1
  index_train=0
  index_val=0
  index_test=0
  for patient in os.listdir(directory_path):
    index_patient=int(patient[-1])
    if index_patient>3:
      index_patient-=1
    for file_path in os.listdir(os.path.join(directory_path, patient)):
      file_name=re.match(r'^(.*?)\.', file_path).group(1)
      image_path=os.path.join(directory_path, patient,file_path)
      label_path=os.path.join(label_directory_path,file_name+'.nrrd')

      image = imageio.v2.imread(image_path)
      image=image[:,:,0:1] #select only one channel, it's a grey image therefore all rgb channels have the same number, pixel by pixel
      image=image/255 #normalize in 0-1

      label_array = sitk.GetArrayFromImage(sitk.ReadImage(label_path))[0, :, :]    #AGGIUSTATO QUALCHE FILE SCRITTO MALE SU SEGMENTED_CHECKED CHE ERANO MESSI MALE. COPIARE LO STESSO DATASET DI QUESTO DRIVE
      binary_images = np.stack([(label_array == level).astype(int) for level in unique_gray_levels], axis=-1)

      i +=1
      if i<i1:
        X_train_list.append(image)
        y_train_list.append(binary_images)
        index_train+=1
      elif i<ii1:
        X_val_list.append(image)
        y_val_list.append(binary_images)
        index_val+=1
      elif i<iii1:
        X_test_list.append(image)
        y_test_list.append(binary_images)
        index_test+=1

      elif i<i2:
        X_train_list.append(image)
        y_train_list.append(binary_images)
        index_train+=1
      elif i<ii2:
        X_val_list.append(image)
        y_val_list.append(binary_images)
        index_val+=1
      elif i<iii2:
        X_test_list.append(image)
        y_test_list.append(binary_images)
        index_test+=1

      elif i<i3:
        X_train_list.append(image)
        y_train_list.append(binary_images)
        index_train+=1
      elif i<ii3:
        X_val_list.append(image)
        y_val_list.append(binary_images)
        index_val+=1
      elif i<iii3:
        X_test_list.append(image)
        y_test_list.append(binary_images)
        index_test+=1

      elif i<i4:
        X_train_list.append(image)
        y_train_list.append(binary_images)
        index_train+=1
      elif i<ii4:
        X_val_list.append(image)
        y_val_list.append(binary_images)
        index_val+=1
      else:
        X_test_list.append(image)
        y_test_list.append(binary_images)
        index_test+=1

  X_train = np.array(X_train_list)
  y_train = np.array(y_train_list)
  X_val = np.array(X_val_list)
  y_val = np.array(y_val_list)
  X_test = np.array(X_test_list)
  y_test = np.array(y_test_list)

  #save memory
  del X_train_list
  del X_test_list
  del X_val_list
  del y_train_list
  del y_test_list
  del y_val_list

  #shuffle them to prevent bias
  X_train, y_train = shuffle(X_train, y_train, random_state=seed)
  X_val, y_val = shuffle(X_val, y_val, random_state=seed)
  X_test, y_test = shuffle(X_test, y_test, random_state=seed)



else:
  #script già caricato

  #Data augmentation *args (not necessary)
  DATA_AUGMENTATION = False
  ROTATION = False
  FLIP = False
  CROP_SIZE = None

  """
  train_ds, validation_ds, test_ds are iterators wich contains:
          1) tensors of shape [BATCH_SIZE, x, y, CHANNELS] for MRI images,
          2) tensors of shape [BATCH_SIZE, x, y, NUM_CLASSES] for labels.

  divided according to the percentages
  """

  train_ds, validation_ds, test_ds = mdl.get_dataset(data_dir = DATASET_PATH,
                                                      percentages = [PERC_TRAIN_DATA, PERC_VALIDATION_DATA, PERC_TEST_DATA],
                                                      batch_size = BATCH_SIZE,
                                                      train_augmentation = DATA_AUGMENTATION,
                                                      random_crop_size = CROP_SIZE,
                                                      random_rotate = ROTATION,
                                                      random_flip = FLIP,
                                                      )


  """
  for simplicity we iterate over the abovementioned iterators to fill the numpy tensors.
  Iterators are faster and more efficient than numpy.array, but to be manipulated (e.g. preprocessing)
  they require high-level python programming, for this reason you will manipulate numpy.array.
  """
  #Empty lists filled from the 3 cicles
  x_train = []
  y_train = []
  x_val = []
  y_val = []
  x_test = []
  y_test = []

  i = 0
  for img, lbl in train_ds.unbatch():
      x_train.append(img)
      y_train.append(lbl)
      i += 1

  for img, lbl in validation_ds.unbatch():
      x_val.append(img)
      y_val.append(lbl)
      i += 1

  for img, lbl in test_ds.unbatch():
      x_test.append(img)
      y_test.append(lbl)
      i += 1

  print('Tot Imgs: {}'.format(i))

  #From list to tensor
  X_train = np.asarray(x_train)
  y_train = np.asarray(y_train)
  X_val = np.asarray(x_val)
  y_val = np.asarray(y_val)
  X_test = np.asarray(x_test)
  y_test = np.asarray(y_test)

  del x_train
  del x_val
  del x_test


#convert to reduce memory usage decrease of about 2Gb
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')


#Some info
print('x_train shape: {}  y_train shape: {}'.format(X_train.shape, y_train.shape))
print('x_val shape: {}  y_val shape: {}'.format(X_val.shape, y_val.shape))
print('x_test shape: {}  y_test shape: {}'.format(X_test.shape, y_test.shape))

x_train shape: (574, 256, 256, 1)  y_train shape: (574, 256, 256, 7)
x_val shape: (163, 256, 256, 1)  y_val shape: (163, 256, 256, 7)
x_test shape: (83, 256, 256, 1)  y_test shape: (83, 256, 256, 7)


# Preprocessing

## Gamma filter

In [ ]:
def gamma_transf(dataset, gamma=0.8):

  x_scaled = (dataset * 255).astype(np.uint8)
  x_processed = np.power(x_scaled / 255.0, gamma)
  x_processed = (x_processed - np.min(x_processed)) / (np.max(x_processed) - np.min(x_processed)) # Scale the pixel values back to the range [0, 1]

  return x_processed

## Gaussian filter + further processing

In [ ]:
def gaussian_filter (dataset,sigma=0.3):

  image = sitk.GetImageFromArray(dataset) #transform array into image class
  image_processed = sitk.SmoothingRecursiveGaussian(image, sigma)
  x_processed = sitk.GetArrayFromImage(image_processed) #get back to array class
  x_processed = (x_processed - np.min(x_processed)) / (np.max(x_processed) - np.min(x_processed)) #normalize between 0 and 1

  return x_processed

In [ ]:
def remove_dots_from_background(dataset, dot_threshold=200):
    # Inizializza una lista per memorizzare le immagini risultanti
    result_images = []

    # Itera attraverso le immagini nel dataset
    for image in dataset:
        # Converti l'immagine in scala di grigi se è a colori
        if len(image.shape) == 3:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray_image = image

        # Applica una soglia per identificare i puntini
        _, binary_image = cv2.threshold(gray_image, dot_threshold, 255, cv2.THRESH_BINARY)

        # Inverti l'immagine binaria (puntini diventano neri)
        inverted_binary_image = cv2.bitwise_not(binary_image)

        # Assicura che la maschera abbia le stesse dimensioni dell'immagine
        if len(image.shape) == 3:
            inverted_binary_image = cv2.cvtColor(inverted_binary_image, cv2.COLOR_GRAY2BGR)

        # Applica l'immagine binaria invertita allo sfondo originale
        result_image = cv2.bitwise_and(image, inverted_binary_image)

        # Aggiungi l'immagine risultante alla lista
        result_images.append(result_image)

    # Converti la lista di immagini in un array numpy
    result_images = np.array(result_images)

    return result_images


## Apply processing

Da fare con il nome dei dati preso da sopra
X_train
X_val
X_test
-> sono le immagini divise ma non ancora processate


In [ ]:
def preprocessing(dataset):
  dataset = gamma_transf(dataset, gamma=1.50)
  dataset = gaussian_filter (dataset, sigma=0.40)
  dataset = remove_dots_from_background(dataset, dot_threshold=200)
  #dataset = reduce_pixelation(dataset, scale_factor=10)
  return dataset

In [ ]:
X_train_denoised = preprocessing(X_train)
X_val_denoised = preprocessing(X_val)
X_test_denoised = preprocessing(X_test)

In [ ]:
print('x_train shape: {}'.format(X_train_denoised.shape))
print('x_val shape: {}'.format(X_val_denoised.shape))

x_train shape: (574, 256, 256)
x_val shape: (163, 256, 256)


In [ ]:
#Stampo due immagini da X_train e X_train_denoised per fare il confronto
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(X_train[0,:,:,0])
plt.title('Original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(X_train_denoised[0,:,:])
plt.title('Processed')
plt.axis('off')

#Stampo due immagini da X_train e X_train_denoised per fare il confronto
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(X_train[0,:,:,0],cmap='gray')
plt.title('Original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(X_train_denoised[0,:,:],cmap='gray')
plt.title('Processed')
plt.axis('off')

Output hidden; open in https://colab.research.google.com to view.

##Calcolo SNR


In [ ]:
def SNR (image, filtered_image):
  # Immagine originale
  original_image = image

  # Immagine dopo il processing
  processed_image = filtered_image

  # Calcola la varianza del segnale nell'immagine originale
  signal_power = np.var(original_image)

  # Calcola la varianza del rumore (differenza tra immagine processata e originale)
  noise_data = (processed_image - original_image)
  noise_power = np.var(noise_data)

  # Calcola l'SNR in dB
  snr = 10 * np.log10(signal_power / noise_power)

  return snr


In [ ]:
#chiamata dell'SNR
snr_list = []

# Itera attraverso le coppie di immagini
for original_image, processed_image in zip(X_train, X_train_denoised):
    # Calcola l'SNR per la coppia di immagini correnti
    snr = SNR(original_image, processed_image)

    # Aggiungi l'SNR alla lista
    snr_list.append(snr)

    # Stampa la lista degli SNR
    print("Lista degli SNR:")
    for snr in snr_list:
        print(snr)


Output streaming troncato alle ultime 5000 righe.
-2.8327277978777667
-2.842396192015075
-2.8845239386070642
-2.8777017475527367
-2.8565378920391393
-2.8583120021996895
-2.8471219585983274
-2.9033624359262618
-2.7773033058058143
-2.802490476376525
-2.6656449112282914
-2.8473737984443765
-2.8134404314478805
-2.830778730609771
-2.624222417318859
-2.8585927324402203
-2.86554259389683
-3.0816941412736343
-2.8127353839703866
-2.6843739794276464
-2.8566329324115958
-2.85928152182509
-2.826264779203174
-2.6282441649624753
-3.0311732518846997
-2.797995509997823
-3.0693393107995415
-3.0248982675784712
-2.8207220619128615
-2.8430489813511812
-2.808372930726382
-2.91931027823427
-2.6765281452628793
-3.071067444595692
-2.8710491585043862
-2.8793404390632555
-2.7970480014425227
-2.666479780759138
-2.726612788639677
-2.8250623275983977
-2.887062203329026
-2.990114902877596
-2.641515739222375
-2.808468863579647
-2.976427879576877
-2.8593563199678584
-2.8561105595196246
-2.607457767027327
-2.820292649

In [ ]:
#cancello x_train, x_val e x_test
del X_train
del X_val
del X_test

# Save variables

In [ ]:
np.save('/content/gdrive/MyDrive/PROJECT_NEURO/Processed_X/X_train_denoised.npy', X_train_denoised)
np.save('/content/gdrive/MyDrive/PROJECT_NEURO/Processed_X/X_val_denoised.npy', X_val_denoised)
np.save('/content/gdrive/MyDrive/PROJECT_NEURO/Processed_X/X_test_denoised.npy', X_test_denoised)

np.save('/content/gdrive/MyDrive/PROJECT_NEURO/y/y_train.npy', y_train)
np.save('/content/gdrive/MyDrive/PROJECT_NEURO/y/y_val.npy', y_val)
np.save('/content/gdrive/MyDrive/PROJECT_NEURO/y/y_test.npy', y_test)